In [4]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Cargamos las tablas normalizadas (según sus ruta)

In [ ]:
items_games = pd.read_parquet('../STEAM-MLOps/items_games.parquet')
reviews_games = pd.read_parquet('../STEAM-MLOps/reviews_games.parquet')

steam_games = pd.read_parquet('../STEAM-MLOps/steam_games_ml.parquet')

Observamos los géneros diferentes

In [5]:
reviews_games['genres'].unique()

array(['Action', 'Indie', 'Simulation', 'Adventure', 'RPG', 'Strategy',
       'Free to Play', 'Massively Multiplayer', 'Casual', 'Early Access',
       'Animation &amp; Modeling', 'Video Production', 'Racing', 'Sports',
       'Audio Production', 'Software Training', 'Utilities',
       'Design &amp; Illustration', 'Web Publishing', 'Education'],
      dtype=object)

In [13]:
steam_games

,item_id,title,description
0,761140,Lost Summoner Kitty,"Action, Casual, Indie, Simulation, Strategy, S..."
1,643980,Ironbound,"Free to Play, Indie, RPG, Strategy, Free to Pl..."
2,670290,Real Pool 3D - Poolians,"Casual, Free to Play, Indie, Simulation, Sport..."
3,767400,弹炸人2222,"Action, Adventure, Casual, Action, Adventure, ..."
4,772540,Battle Royale Trainer,"Action, Adventure, Simulation, Action, Adventu..."
...,...,...,...
28233,745400,Kebab it Up!,"Action, Adventure, Casual, Indie, Action, Indi..."
28234,773640,Colony On Mars,"Casual, Indie, Simulation, Strategy, Strategy,..."
28235,733530,LOGistICAL: South Africa,"Casual, Indie, Strategy, Strategy, Indie, Casu..."
28236,610660,Russian Roads,"Indie, Racing, Simulation, Indie, Simulation, ..."


## Procedemos con la librería **pandas**

### Funciones

In [6]:
def PlayTimeGenre(genero: str):
    '''
    devuelve el año con mas horas jugadas para dicho género

    Args: 
        genero (str): género del juego

    return:
        dict: diccionario con el género X y el año de lanzamiento con más horas jugadas
    
    '''
    # Filtramos el dataframe 'items_games' respecto al parámetro genero
    df_util = items_games[items_games['genres']== genero]
    
    # Agrupamos el dataframe anterior por año de lanzamiento, suma de minutos de juego y ordenamos en forma descendente
    agrupado = df_util.groupby('year_release')['playtime_forever'].sum().sort_values(ascending=False)

    # El valor máximo tendrá índice [0]
    anio = agrupado.index[0]
    
    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": int(anio)}

In [7]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

In [8]:
def UserForGenre(genero: str):
    '''
    devuelve el usuario con mas horas jugadas para dicho género

    Args: 
        genero (str): género del juego

    return:
        dict: diccionario con el género X y el usuario con más horas jugadas
    
    '''
    # Filtramos el dataframe 'items_games' respecto al parámetro genero
    df_util = items_games[items_games['genres']== genero]

    # Agrupamos el dataframe anterior por usuario, suma de minutos de juego y ordenamos en forma descendente
    agrupado = df_util.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)

    # El valor máximo tendrá índice [0]
    user = agrupado.index[0]

    # tomamos las filas del dataframe util que contengan su repsectivo usuario (user)
    df_genero_user = df_util[df_util['user_id']==user]

    # agrupamos respecto a los años y suma de minutos de juego y convertimos a horas de juego
    horas_jugadas = round(df_genero_user.groupby('year_release')['playtime_forever'].sum()/60, 3)

    # Guardamos la serie 'horas_jugadas' en una lista
    lista_horas_jugadas = [f'Año: {int(anio)}, Horas: {horas}' for anio, horas in horas_jugadas.items()]

    return {f"Usuario con más horas jugadas para género {genero}": user, "Horas jugadas": lista_horas_jugadas}

In [9]:
UserForGenre('Action')

{'Usuario con más horas jugadas para género Action': 'REBAS_AS_F-T',
 'Horas jugadas': ['Año: 1996, Horas: 0.0',
  'Año: 1998, Horas: 0.033',
  'Año: 1999, Horas: 3.75',
  'Año: 2000, Horas: 0.0',
  'Año: 2001, Horas: 0.183',
  'Año: 2002, Horas: 0.017',
  'Año: 2003, Horas: 31.05',
  'Año: 2004, Horas: 35.25',
  'Año: 2005, Horas: 54.883',
  'Año: 2006, Horas: 2.3',
  'Año: 2007, Horas: 6.25',
  'Año: 2008, Horas: 42.883',
  'Año: 2009, Horas: 132.1',
  'Año: 2010, Horas: 74.333',
  'Año: 2011, Horas: 628.417',
  'Año: 2012, Horas: 843.917',
  'Año: 2013, Horas: 1626.1',
  'Año: 2014, Horas: 2635.233',
  'Año: 2015, Horas: 2707.533',
  'Año: 2016, Horas: 2309.533',
  'Año: 2017, Horas: 33.167']}

In [10]:
def UserRecommend(anio: int):
    '''
    devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    
    Args: 
        anio (int): año de consulta

    return:
        dict: diccionario con los 3 juegos más recomendados por usuarios
        ->  reviews['recommend'] = True(1)
        ->  comentarios positivos(2) o neutros(1)

    '''
    # Si el año de lanzamiento(year_release) no coincide con alguno de los años en los que se hace una reseña(year_posted), se retorna un mensaje de erro
    if anio not in reviews_games['year_posted'].unique():
        return f"Año fuera de rango, ingrese un año válido"
    
    else:
        # Filtramos el dataframe con las filas cuyo año de posteo(year_posted) es mayor o igual al año de publicación(year_release)
        df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]
        
        # Filtramos el dataframe 'df' para el año parámetro y la columna sentiment_analysis sea positivo(2) o neutro(1)
        df_anio_dado = df[(df['year_posted']==anio) & (df['sentiment_analysis'].isin([1,2]))]

        # Agrupamos el dataframe 'df_anio_dado' por título del juego ('title'), sumamos las recomendaciones('recommend') para tener los juegos más recomendados y ordenamos de forma descendente
        top = df_anio_dado.groupby('title')['recommend'].sum().sort_values(ascending=False)

        # Construimos el top3
        top3 = [{"Puesto 1": top.index[0]}, {"Puesto 2": top.index[1]}, {"Puesto 3": top.index[2]}]

    return top3

In [11]:
UserRecommend(2012)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Terraria'},
 {'Puesto 3': 'Realm of the Mad God'}]

In [12]:
def UsersNotRecommend(anio: int):
    '''
    devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
    
    Args: 
        anio (int): año de consulta

    return:
        dict: diccionario con los 3 juegos menos recomendados por usuarios
        ->  reviews['recommend'] = False(0)
        ->  comentarios negativos(reviews['sentiment_analysis']==0)

    '''
    # Si el año de lanzamiento(year_release) no coincide con alguno de los años en los que se hace una reseña(year_posted), se retorna un mensaje de erro
    if anio not in reviews_games['year_posted'].unique():
        return f"Año fuera de rango, ingrese un año válido"
    
    else:
        # Filtramos el dataframe con las filas cuyo año de posteo(year_posted) es mayor o igual al año de publicación(year_release)
        df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]

        #Ahora filtramos por el año parámetro, recomendaciones negativas(0) y comentarios negativos (0)
        df_anio_dado = df[(df['year_posted']==anio) & (df['recommend']==0) & (df['sentiment_analysis']==0)]

        # Agrupamos respecto al año título del juego, contamos las recomendaciones('recommend') para tener los juegos con más reseñas negtivas y ordenamos de forma descendente
        grupo = df_anio_dado.groupby('title')['recommend'].count().sort_values(ascending=False)

        # Contruimos el top3
        top3 = [{"Puesto 1": grupo.index[0]}, {"Puesto 2": grupo.index[1]}, {"Puesto 3": grupo.index[2]}]

        return top3

In [13]:
UsersNotRecommend(2013)

[{'Puesto 1': 'Half-Life: Before'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': 'Tactical Intervention'}]

In [14]:
def sentiment_analysis(anio: int):
    '''
    Según el año de lanzamiento, devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento

    Args: 
        anio (int): año de lanzamiento (year_release)

    return:
        list: lista con la cantidad de registros de reseñas de usuarios categorizados

    '''
    # Si el año de lanzamiento(year_release) no coincide con alguno de los años en los que se hace una reseña(year_posted), se retorna un mensaje de erro
    if anio not in reviews_games['year_posted'].unique():
        return f"Año fuera de rango, ingrese un año válido"
    
    else:
        # Filtramos el dataframe con las filas cuyo año de posteo(year_posted) es mayor o igual al año de publicación(year_release)
        df = reviews_games[reviews_games['year_posted']>=reviews_games['year_release']]

        # Filtramos el dataframe 'df' para el año parámetro
        df_anio = df[df['year_release'] == anio]

        # Contamos las filas del dataframe 'df_anio' respecto a los valores únicos de la columna 'sentiment_analysis' {0,1,2} y los guardamos en sus respectivos sentimientos
        positivos = df_anio[df_anio['sentiment_analysis']==2].shape[0] # número de filas de sentimientos positivos(2)
        neutros = df_anio[df_anio['sentiment_analysis']==1].shape[0] # número de filas de sentimientos nneutros(1)
        negativos = df_anio[df_anio['sentiment_analysis']==0].shape[0] # número de filas de sentimientos negativos(0)

        return {'Negative': negativos, 'Neutral': neutros, 'Positive': positivos}

In [15]:
sentiment_analysis(2012)

{'Negative': 2372, 'Neutral': 2843, 'Positive': 9733}

### Modelo 1

Utilizamos la clase `CountVectorizer` de scikit para convertir las descripciones en representaciones numéricas y el método `cosine_similarity` para evaluar la similitud coseno de las descripciones

In [12]:
import psutil

# Creamos una instancia de la clase CountVectorizer
vector = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Dividimos cada cadena de descripción en palabras individuales y se crea una matriz de conteo 'matriz_descripcion' que representa cuántas veces aparece cada género en cada videojuego.
matriz_descripcion = vector.fit_transform(steam_games['description'])

def recomendacion_juego(id_producto: int):
    '''
    Se ingresa el id de producto (item_id) y retorna una lista con 5 juegos recomendados similares al ingresado (title).
    
    '''
    # Si el id ingresado no se encuentra en la columna de id de la tabla 'steam_games' se le pide al usuario que intente con otro id
    if id_producto not in steam_games['item_id'].values:
        return 'El ID no existe, intente con otro'
    else:
        # buscamos el índice del id ingresado
        index = steam_games.index[steam_games['item_id']==id_producto][0]

        # De la matriz de conteo, tomamos el array de descripciones con índice igual a 'index'
        description_index = matriz_descripcion[index]

        # Calculamos la similitud coseno entre la descripción de entrada y la descripción de las demás filas: cosine_similarity(description_index, matriz_descripcion)
        # Obtenemos los índices de las mayores similitudes mediante el método argsort() y las similitudes ordenadas de manera descendente
        # Tomamos los índices del 1 al 6 [0, 1:6] ya que el índice 0 es el mismo índice de entrada
        indices_maximos = np.argsort(-cosine_similarity(description_index, matriz_descripcion))[0, 1:6]

        # Construimos la lista
        recomendaciones = []
        for i in indices_maximos:
            recomendaciones.append(steam_games['title'][i])
        
        return recomendaciones

print(psutil.Process().memory_info().rss)

994594816


C:\Users\PC - Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
recomendacion_juego(745400)

['Dark Snow',
 'M1: A Death in the Desert',
 'The Moon Night',
 'Swingin Swiggins',
 'Cyborg Arena']